In [1]:
!wget https://huggingface.co/datasets/bezirganyan/LUMA/resolve/main/text_data.tsv

--2024-11-25 20:08:59--  https://huggingface.co/datasets/bezirganyan/LUMA/resolve/main/text_data.tsv
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/91/6d/916d91190e01032d29384546038721047bf54420deab1052a661200d4b695d3d/34fdb88e6c762deda6bc625f7eb51394a584ff9306a9d7753ab223859754e42a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27text_data.tsv%3B+filename%3D%22text_data.tsv%22%3B&response-content-type=text%2Ftab-separated-values&Expires=1732824539&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjgyNDUzOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzkxLzZkLzkxNmQ5MTE5MGUwMTAzMmQyOTM4NDU0NjAzODcyMTA0N2JmNTQ0MjBkZWFiMTA1MmE2NjEyMDBkNGI2OTVkM2QvMzRmZGI4OGU2Yzc2MmRlZGE2YmM2MjVmN2ViNTEzOTRhNTg0ZmY5MzA

In [23]:
import pandas as pd
import random
from tqdm import tqdm

In [2]:
data = pd.read_csv('text_data.tsv', sep='\t')

In [8]:
print(f'lenght of data: {len(data)}')

lenght of data: 62875


In [10]:
idx = random.randint(0, len(data))
print(f'sample {idx} \nText: {data.loc[idx].text} \nLabel: {data.loc[idx].label}')

sample 19656 
Text: Castles have been depicted in countless works of literature, film, and television. They are often portrayed as imposing structures of mystery, power, and grandeur. In literature, castles are frequently used as settings for stories of adventure, romance, and tragedy. In film and television, castles are used to create a sense of spectacle and awe. The word "castle" is also often used figuratively to describe anything that is tall, imposing, or majestic. 
Label: castle


## Testing some models

We first filter the biases from non biased

In [41]:
# @title Binary classifier
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device=0) # cuda = 0,1 based on gpu availability

Some layers from the model checkpoint at d4data/bias-detection-model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at d4data/bias-detection-model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
# test on one sample

output = classifier(data.loc[idx].text),

In [43]:
output[0],

([{'label': 'Biased', 'score': 0.9843871593475342}],)

In [44]:
data.loc[idx].text

'Castles have been depicted in countless works of literature, film, and television. They are often portrayed as imposing structures of mystery, power, and grandeur. In literature, castles are frequently used as settings for stories of adventure, romance, and tragedy. In film and television, castles are used to create a sense of spectacle and awe. The word "castle" is also often used figuratively to describe anything that is tall, imposing, or majestic.'

In [45]:
data['text_length'] = data['text'].str.len()
data_sorted = data.sort_values(by='text_length')

In [48]:
# Batching the input
batch_size = 64  # Adjust based on your GPU memory
results = []
for i in tqdm(range(0, len(data_sorted), batch_size)):
    batch = data_sorted['text'][i:i + batch_size].tolist()
    try:
        batch_results = classifier(batch)
        results.extend(batch_results)
    except Exception as e:
        print(f"Error processing batch {i}: {e}")

 20%|██        | 199/983 [21:34<1:24:59,  6.50s/it]


KeyboardInterrupt: 

In [ ]:
# @title Multiclass classification
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="PriyaPatel/bias_identificaiton45")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
classifier(data.loc[idx].text)

[{'label': 'Biased', 'score': 0.9555160403251648}]

In [ ]:
data.loc[idx].text

"The wind whips through your hair,\nA ball of fur, with energy to spare.\nYour tail wags with every beat of your heart,\nA melody of joy, a vibrant start.\n\nYour eyes shine like golden light,\nCleansing the darkest night.\nIn every moment, a pure delight,\nYour presence fills me with delight.\n\nYour bark, a warning, a protective sound,\nA guardian's call, known throughout the ground.\nYou chase, play, and bring me cheer,\nThe best companion, year after year.\n\nYour energy is contagious,\nYou fill me with zest and motivation.\nI want to be like you, full of grace,\nYour spirit ignites, a warm, cozy space.\n\n A dog is a loyal companion, full of joy, with a heart that can bring happiness to all. Their boundless energy, fierce protectiveness, and unconditional love make them the best pets anyone could ask for."